<a href="https://colab.research.google.com/github/Dexheimer/cotacao_ceasa/blob/main/Precos_Ceasa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pdfplumber # Install the pdfplumber library using pip
import os
import requests
import pandas as pd
from PyPDF2 import PdfReader
import pdfplumber # Now you should be able to import pdfplumber without any issues

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 74.3 MB/s eta 0:00:00


In [5]:
!pip install requests beautifulsoup4 pandas pdfplumber

In [6]:
import requests
from bs4 import BeautifulSoup
import os

# URL da página que lista os PDFs
url_base = "https://www.rj.gov.br/ceasa/node/387"

# Diretório para salvar os PDFs
output_dir = "ceasa_pdfs"
os.makedirs(output_dir, exist_ok=True)

# Requisição para obter o HTML da página
response = requests.get(url_base)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Busca todas as tags de link para PDFs
    links = soup.find_all('a', href=True)
    pdf_links = [link['href'] for link in links if link['href'].endswith('.pdf')]

    print(f"Encontrados {len(pdf_links)} PDFs. Iniciando o download...")

    # Download dos PDFs
    for pdf_link in pdf_links:
        pdf_url = pdf_link if pdf_link.startswith('http') else f"https://www.rj.gov.br{pdf_link}"
        file_name = os.path.basename(pdf_url)
        file_path = os.path.join(output_dir, file_name)

        # Faz o download
        pdf_response = requests.get(pdf_url)
        if pdf_response.status_code == 200:
            with open(file_path, "wb") as file:
                file.write(pdf_response.content)
            print(f"Baixado: {file_name}")
        else:
            print(f"Falha no download: {pdf_url}")
else:
    print("Erro ao acessar a página principal.")

Encontrados 22 PDFs. Iniciando o download...
Baixado: Boletim%20de%20pre%C3%A7os%20%2031%2001%202024_0.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2030%2001%202024_0.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2029%2001%202024_0.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2026%2001%202024_0.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2025%2001%202024_0.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2024%2001%202024_0.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2023%2001%202024_0.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2022%2001%202024.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2019%2001%202024.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2018%2001%202024.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2017%2001%202024.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2016%2001%202024.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2015%2001%202024.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2012%2001%202024.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%2011%2001%202024.pdf
Baixado: Boletim%20de%20pre%C3%A7os%20%20

In [7]:
import pdfplumber
import pandas as pd

# Lista para armazenar os dados
data = []

# Processar os PDFs
for pdf_file in os.listdir(output_dir):
    if pdf_file.endswith(".pdf"):
        with pdfplumber.open(os.path.join(output_dir, pdf_file)) as pdf:
            for page in pdf.pages:
                table = page.extract_table()
                if table:
                    # Adiciona cada linha da tabela ao conjunto de dados
                    for row in table:
                        data.append(row)

# Criação do DataFrame
df = pd.DataFrame(data)

# Visualizar primeiros registros
print(df.head())

# Salvando como CSV
df.to_csv("ceasa_prices.csv", index=False, encoding="utf-8-sig")
print("Dados salvos no arquivo ceasa_prices.csv")



                     0            1                  2  \
0                 None  Dia Semana:      segunda-feira   
1             PRODUTOS         TIPO  UNIDADE EMBALAGEM   
2  1. FRUTAS NACIONAIS                                   
3              ABACATE                        Cx 15 kg   
4       ABACAXI ANANÁS       Grande        Unid 2,5 kg   

                             3      4      5           6  
0                         None   None         15/01/2024  
1  VARIAÇÃO\nULTIMOS\n12 MESES    MIN  MODAL         MAX  
2                                                         
3                      -17,65%  60,00  70,00       80,00  
4                          S/C                            
Dados salvos no arquivo ceasa_prices.csv


In [14]:
print(df.head())

                     0            1                  2  \
0                 None  Dia Semana:      segunda-feira   
1             PRODUTOS         TIPO  UNIDADE EMBALAGEM   
2  1. FRUTAS NACIONAIS                                   
3              ABACATE                        Cx 15 kg   
4       ABACAXI ANANÁS       Grande        Unid 2,5 kg   

                             3      4      5           6  
0                         None   None         15/01/2024  
1  VARIAÇÃO\nULTIMOS\n12 MESES    MIN  MODAL         MAX  
2                                                         
3                      -17,65%  60,00  70,00       80,00  
4                          S/C                            


In [15]:
# Listar os nomes das colunas
print("Nomes das colunas no DataFrame:")
print(df.columns)


Nomes das colunas no DataFrame:
RangeIndex(start=0, stop=7, step=1)


In [17]:
# Lista com os nomes das colunas (substitua pelos nomes reais)
novos_nomes = ["Produto", "Obs", "Unidade Embalagem", "Variação últimos 12 meses", "Preço_Mínimo", "Preço_Médio", "Preço_Máximo"]

# Substituir os nomes das colunas no DataFrame
df.columns = novos_nomes

# Verificar os novos nomes
print("Novos nomes das colunas:")
print(df.columns)


Novos nomes das colunas:
Index(['Produto', 'Obs', 'Unidade Embalagem', 'Variação últimos 12 meses',
       'Preço_Mínimo', 'Preço_Médio', 'Preço_Máximo'],
      dtype='object')


In [18]:
# Listar os nomes das colunas
print("Nomes das colunas no DataFrame:")
print(df.columns)

Nomes das colunas no DataFrame:
Index(['Produto', 'Obs', 'Unidade Embalagem', 'Variação últimos 12 meses',
       'Preço_Mínimo', 'Preço_Médio', 'Preço_Máximo'],
      dtype='object')


In [19]:
# Salvar o DataFrame atualizado em um novo arquivo CSV
df.to_csv("ceasa_prices_renomeado.csv", index=False, encoding="utf-8-sig")

print("Novo arquivo CSV criado: ceasa_prices_renomeado.csv")

Novo arquivo CSV criado: ceasa_prices_renomeado.csv


In [20]:
# Lista com os termos a serem excluídos

termos_excluir = ["PRODUTOS", "TIPO", "UNIDADE", "EMBALAGEM",
    "VARIAÇÃO ULTIMOS 12 MESES", "MIN", "MODAL", "MAX"]

In [21]:
# Filtrar as linhas que contenham qualquer um dos termos da lista

mascara = df.apply(lambda row: row.astype(str).str.contains('I'.join(termos_excluir), case=False).any(), axis=1)

In [23]:
# Salvar o DataFrame sem as linhas indesejadas
df.to_csv("ceasa_prices_filtrado.csv", index=False, encoding="utf-8-sig")

print("Novo arquivo CSV criado: ceasa_prices_filtrado.csv")


Novo arquivo CSV criado: ceasa_prices_filtrado.csv


In [24]:

# Lista com os termos a serem excluídos
termos_excluir = [
    "PRODUTOS", "TIPO", "UNIDADE", "EMBALAGEM",
    "VARIAÇÃO ULTIMOS 12 MESES", "MIN", "MODAL", "MAX"
]

# Aplicar o filtro para todas as células do DataFrame
mascara = df.apply(lambda row: row.astype(str).str.contains('|'.join(termos_excluir), case=False).any(), axis=1)

# Excluir as linhas que contêm qualquer um dos termos da lista
df_filtrado = df[~mascara]

# Verificar o DataFrame resultante
print(df_filtrado.head())

# Salvar o DataFrame filtrado em um novo arquivo CSV (opcional)
df_filtrado.to_csv("ceasa_prices_filtrado.csv", index=False, encoding="utf-8-sig")
print("Novo arquivo CSV criado: ceasa_prices_filtrado.csv")

               Produto          Obs Unidade Embalagem  \
0                 None  Dia Semana:     segunda-feira   
2  1. FRUTAS NACIONAIS                                  
3              ABACATE                       Cx 15 kg   
4       ABACAXI ANANÁS       Grande       Unid 2,5 kg   
5       ABACAXI PÉROLA        Médio       Unid 2,0 kg   

  Variação últimos 12 meses Preço_Mínimo Preço_Médio Preço_Máximo  
0                      None         None               15/01/2024  
2                                                                  
3                   -17,65%        60,00       70,00        80,00  
4                       S/C                                        
5                    24,82%         7,00        8,00         8,50  
Novo arquivo CSV criado: ceasa_prices_filtrado.csv


In [25]:
# Criar uma nova coluna 'Data' copiando os valores de 'Preço_Máximo'
df['Data'] = df['Preço_Máximo']

In [26]:
import re

# Função para verificar se uma string é uma data no formato DD MM AAAA
def is_date(value):
    return bool(re.match(r"\d{2} \d{2} \d{4}", str(value)))

# Filtrar a coluna 'Data' e manter apenas as células que são datas
df['Data'] = df['Data'].apply(lambda x: x if is_date(x) else None)

# Verificar as primeiras linhas para ver como ficou
print(df.head())

               Produto          Obs  Unidade Embalagem  \
0                 None  Dia Semana:      segunda-feira   
1             PRODUTOS         TIPO  UNIDADE EMBALAGEM   
2  1. FRUTAS NACIONAIS                                   
3              ABACATE                        Cx 15 kg   
4       ABACAXI ANANÁS       Grande        Unid 2,5 kg   

     Variação últimos 12 meses Preço_Mínimo Preço_Médio Preço_Máximo  Data  
0                         None         None               15/01/2024  None  
1  VARIAÇÃO\nULTIMOS\n12 MESES          MIN       MODAL          MAX  None  
2                                                                     None  
3                      -17,65%        60,00       70,00        80,00  None  
4                          S/C                                        None  


In [27]:
# Preencher as células vazias na coluna 'Data' com a data anterior
df['Data'] = df['Data'].fillna(method='ffill')

# Verificar as primeiras linhas para garantir que as datas foram preenchidas
print(df.head())


               Produto          Obs  Unidade Embalagem  \
0                 None  Dia Semana:      segunda-feira   
1             PRODUTOS         TIPO  UNIDADE EMBALAGEM   
2  1. FRUTAS NACIONAIS                                   
3              ABACATE                        Cx 15 kg   
4       ABACAXI ANANÁS       Grande        Unid 2,5 kg   

     Variação últimos 12 meses Preço_Mínimo Preço_Médio Preço_Máximo  Data  
0                         None         None               15/01/2024  None  
1  VARIAÇÃO\nULTIMOS\n12 MESES          MIN       MODAL          MAX  None  
2                                                                     None  
3                      -17,65%        60,00       70,00        80,00  None  
4                          S/C                                        None  


<ipython-input-27-c46e2a13ca87>:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Data'] = df['Data'].fillna(method='ffill')


In [28]:
# Salvar o DataFrame atualizado em um novo arquivo CSV
df.to_csv("ceasa_prices_com_data.csv", index=False, encoding="utf-8-sig")

print("Novo arquivo CSV criado: ceasa_prices_com_data.csv")

Novo arquivo CSV criado: ceasa_prices_com_data.csv


In [29]:
!git init
!git config --global user.email "andre.dexheimer@gmail.com"
!git config --global user.name "Dex"
!git add .
!git commit -m "First commit"
!git remote add origin <https://github.com/Dexheimer/cotacao_ceasa.git>
!git push -u origin master

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
[master (root-commit) de48dd6] First commit
 47 files changed, 74655 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce


In [32]:
!git init
!git config --global user.email "andre.dexheimer@gmail.com"
!git config --global user.name "Dex"
!git add .
!git commit -m "First commit"
!git remote add origin <https://github.com/Dexheimer/cotacao_ceasa.git>
!git push -u origin master

Reinitialized existing Git repository in /content/.git/
On branch master
nothing to commit, working tree clean
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `git remote add origin <https://github.com/Dexheimer/cotacao_ceasa.git>'
fatal: could not read Username for 'https://github.com': No such device or address


In [37]:
!git add
!git commit -m "First commit"
!git remote add origin git@github.com:Dexheimer/cotacao_ceasa.git  # Se estiver usando SSH
!git push -u origin master




Nothing specified, nothing added.
hint: Maybe you wanted to say 'git add .'?
hint: Turn this message off by running
hint: "git config advice.addEmptyPathspec false"
On branch master
nothing to commit, working tree clean
error: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address
